In [1]:
from pxr import Usd,Sdf,Ar
import glob
import shutil
import os

Asset

In [2]:
# Assetの置き場所
ASSETS_DIR = "D:/sample/assets"

In [3]:
assetName = "sampleAssets"
version = 1.0
# AssetNameのGeomのVariantにするusda置き場を指定する
geomVariantDir = "D:/sample/variant"
assetDir = f"{ASSETS_DIR}/{assetName}"

In [12]:
payloadLayer = Sdf.Layer.FindOrOpen(f"{assetDir}/{assetName}.payload.usd")
pyaloadStage = Usd.Stage.Open(payloadLayer)

In [4]:
# 実際に読み込むための AssetName.usd を作る
# AssetNameのPrimを作り、それをDefaultPrimに指定する
payloadStage = Usd.Stage.CreateInMemory()
payloadsLayer = payloadStage.GetRootLayer()
prim = payloadStage.DefinePrim(f'/{assetName}')
payloadStage.SetDefaultPrim(prim)
rootPath = prim.GetPath()

In [6]:
# Variantを指定する
vset = prim.GetVariantSets().AddVariantSet('assets')
geomPath = rootPath.AppendChild('Geom')
geomPrim = payloadStage.DefinePrim(geomPath)
payloadStage.DefinePrim(geomPath)

variantDir = f"{assetDir}/variant"
os.makedirs(variantDir,exist_ok=True)

# 指定フォルダ以下にあるレイヤーを、VariantSetとして追加する
for f in glob.glob(f"{geomVariantDir}/*.usd*"):
    # Assets以下にコピーする
    basename = os.path.basename(f)
    shutil.copy2(f,f"{variantDir}/{basename}")
    variantName = os.path.splitext(basename)[0]
    vset.AddVariant(variantName)
    # モデルパターンの切り替え用のVariantSetを作る
    vset.SetVariantSelection(variantName)
    with vset.GetVariantEditContext():
        # variant 以下のモデルをReferenceする
        geomPrim.GetReferences().AddReference(f"./variant/{basename}")
        
payloadsLayer.Export(f"{assetDir}/{assetName}.payload.usd")

True

In [7]:
# Inheritsを作る
inheritsStage = Usd.Stage.CreateInMemory()
inheritsLayer = inheritsStage.GetRootLayer()
inheritsStage.CreateClassPrim("/__class__")
inheritsLayer.Export(f"{assetDir}/{assetName}.class.usd")

True

In [9]:
# Assetsのレイヤーをつくる
stage = Usd.Stage.CreateInMemory()
layer = stage.GetRootLayer()
# そして、AssetsのRoodPrimを作成する
prim = stage.DefinePrim(f'/{assetName}')
rootPath = prim.GetPath()
# Kindを指定
Usd.ModelAPI(prim).SetKind('component')
stage.SetDefaultPrim(prim)

# assetName.usd にはAssetInfoを指定する
assetInfo = {
    "identifier":f"./{assetName}/{assetName}.usd",
    "name":assetName,
    'version':version
}
prim.SetAssetInfo(assetInfo)
# Variantを入れた payload Layer を Payloadする
prim.GetPayloads().AddPayload(f"{assetDir}/{assetName}.payload.usd")

# Inheritsの構造を仕込む
layer.subLayerPaths = [f'./{assetName}.class.usd']
prim.GetInherits().AddInherit('/__class__')


True

In [11]:
# AssetのExport
layer.Export(f"{assetDir}/{assetName}.usd")

True